In [167]:
# importing the necessary packages
import cv2
import dlib
import numpy as np
from renderFace import renderFace
from dataPath import DATA_PATH

In [168]:
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (6.0,6.0)
matplotlib.rcParams['image.cmap'] = 'gray'

In [169]:
# initialize dlib's face detector (HOG-based) and then create the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
# Dlib shape predictor model path
MODEL_PATH = DATA_PATH + "/models/shape_predictor_68_face_landmarks.dat"
# Load model
shape_predictor = dlib.shape_predictor(MODEL_PATH)

[INFO] loading facial landmark predictor...


TODO : Complete the smile detector function
You need to apply the face detector and shape_predictor to the input image to get the landmarks.

Then, Use the landmarks and come up with a logic so that the smile is detected for each frame.

Finally, override the variable isSmiling to True if smile is detected.

Some hints:

- Find the lip and/or jaw coordinates using the facial landmarks.
- For a person to be smiling, the ratio of width of the lip and the jaw width should be high.
- Return True if a smile is detected, else return False.

In [170]:
def smile_detector1(imDlib):
    global prevLandmarks
    # Detect faces
    faces = detector(imDlib, 0)
    
    if len(faces):
        landmarks = shape_predictor(imDlib, faces[0])
    else:
        return False
    
    isSmiling = False

    # Check if the person is smiling
    # landmarkParts = landmarks.parts()
    # lipLeft = (landmarkParts[48].x,landmarkParts[48].y)
    # lipRight = (landmarkParts[54].x,landmarkParts[54].y)
    # jawLeft = (landmarkParts[4].x,landmarkParts[4].y)
    # jawRight = (landmarkParts[14].x,landmarkParts[14].y)

    # lip_distance = cv2.norm(np.array(lipLeft) - np.array(lipRight))
    # jaw_distance = cv2.norm(np.array(jawLeft) - np.array(jawRight))

    # print("Lip distance: ", lip_distance)
    # print("Jaw distance: ", jaw_distance)

    lip_indices = list(range(48, 61))
    jaw_indices = list(range(0, 17))
    threshold = 3.0

    print("Prev Landmarks None: " + str(prevLandmarks is None))
    if prevLandmarks is not None:
        landmarkPoints = np.array([(p.x, p.y) for p in landmarks.parts()])
        prevLandmarkPoints = np.array([(p.x, p.y) for p in prevLandmarks.parts()])
        diffLip = np.linalg.norm(landmarkPoints[lip_indices] - prevLandmarkPoints[lip_indices], axis=1)
        diffJaw = np.linalg.norm(landmarkPoints[jaw_indices] - prevLandmarkPoints[jaw_indices], axis=1)
        isLipMove = np.any(diffLip > threshold)
        isJawMove = np.any(diffJaw > threshold)
        print("Lip Move: ", isLipMove)
        print("Jaw Move: ", isJawMove)
        if(isLipMove and isJawMove):
            isSmiling = True

    prevLandmarks = landmarks
    
    return isSmiling

def smile_detector(imDlib):
    # Detect faces
    faces = detector(imDlib, 0)
    
    if len(faces):
        landmarks = shape_predictor(imDlib, faces[0])
    else:
        return False
    
    isSmiling = False

    threshold = 0.5

    landmarkPoints = np.array([(p.x, p.y) for p in landmarks.parts()])


    lip_left = np.array(landmarkPoints[48])
    lip_right = np.array(landmarkPoints[54])
    jaw_left = np.array(landmarkPoints[5])
    jaw_right = np.array(landmarkPoints[13])
    
    lip_width = np.linalg.norm(lip_right - lip_left)
    jaw_width = np.linalg.norm(jaw_right - jaw_left)
   
    ratio = lip_width / jaw_width

    if ratio > threshold:
        isSmiling = True
    isSmiling = ratio > threshold

    return isSmiling

In [172]:
# Initializing video capture object.
capture = cv2.VideoCapture(DATA_PATH +"videos/smile.mp4")
if(False == capture.isOpened()):
    print("[ERROR] Video not opened properly")    

# Create a VideoWriter object
smileDetectionOut = cv2.VideoWriter(DATA_PATH+"smileDetectionOutput.avi",
                                   cv2.VideoWriter_fourcc('M','J','P','G'),
                                   15,(int(capture.get(cv2.CAP_PROP_FRAME_WIDTH)), 
                                       int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    
frame_number = 0
smile_frames = []
while (True):
    # grab the next frame
    isGrabbed, frame = capture.read()
    if not isGrabbed:
        break
        
    imDlib = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_has_smile = smile_detector(imDlib)

    if (True == frame_has_smile):
        cv2.putText(frame, "Smiling :)", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2, cv2.LINE_AA)
        smile_frames.append(frame_number)
        # cv2.imwrite("image"+str(frame_number)+".jpg", frame)
        # print("Smile detected in Frame# {}".format(frame_number))
    if frame_number % 50 == 0:
        print('\nProcessed {} frames'.format(frame_number))
        print("Smile detected in Frames: {}".format(smile_frames))
    # Write to VideoWriter
    # smileDetectionOut.write(frame)
    
    frame_number += 1

capture.release()
smileDetectionOut.release()


Processed 0 frames
Smile detected in Frames: []

Processed 50 frames
Smile detected in Frames: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

Processed 100 frames
Smile detected in Frames: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

Processed 150 frames
Smile detected in Frames: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 